In [17]:
import numpy as np
import pyuvdata
import newcal
import importlib
importlib.reload(newcal)
from newcal import calibration
importlib.reload(calibration)

<module 'newcal.calibration' from '/Users/ruby/Astro/newcal/newcal/calibration.py'>

In [2]:
data = pyuvdata.UVData()
data.read("/Users/ruby/Astro/test_calibration_Jul2023/20230309_225134_73MHz.ms")

In [3]:
model = pyuvdata.UVData()
model.read("/Users/ruby/Astro/test_calibration_Jul2023/20230309_225134_73MHz_model.uvfits")

In [4]:
(
    gains_init,
    Nants,
    Nbls,
    Ntimes,
    Nfreqs,
    model_visibilities,
    data_visibilities,
    visibility_weights,
    gains_exp_mat_1,
    gains_exp_mat_2,
) = calibration.uvdata_calibration_setup(
    data,
    model,
    gain_init_calfile=None,
    gain_init_stddev=0.0,
    N_feed_pols=2,
)

In [ ]:
gains_fit = calibration.run_calibration_optimization_per_pol(
    gains_init,
    Nants,
    Nbls,
    Nfreqs,
    2,
    model_visibilities,
    data_visibilities,
    visibility_weights,
    gains_exp_mat_1,
    gains_exp_mat_2,
    0,
    xtol=1e-5,
    verbose=False,
)

1937893217703.7556
1696632395682.5503
6872028885658.528
1148242771502.825
1032516754496.1654
1029087651165.7277
1028558222886.7548
946639711880.1655
933250084038.8623
916879805236.9429
905991132088.9127
905708469796.148
905172950465.7717
904252816925.3635
903333851246.3823
903316132558.3108
903281268376.14
903160400707.5151
902966553344.5464
902962361112.1975
902954119173.3092
902925685226.5657
902879991896.707
902879261544.098
902877808787.3877
902872450687.8494
902872030394.9976
902871221237.5482
902868782046.6771
902862460119.0009
902842315061.3433
902838918919.3716
902838538159.8374
902837824252.098
902836914091.1132
902835111970.9304
902828639486.5367
902828163608.7769
902827308892.0524
902825605643.6753
902819262126.2275
902818940953.7727
902818294895.4407
902815764914.8212
902806528821.3263
902805873206.0647
902805504172.8481
902804832758.072
902804792022.3846
902804709748.758
902804380903.4885
902803069510.2078
902797887742.0393
902778181561.7505
902778142749.2825
902778064388.

In [ ]:
np.max(visibility_weights[:, :, 0, 0,])

In [ ]:
freq_ind = 0
pol_ind = 0
gains_init_flattened = np.stack(
    (
        np.real(gains_init[:, freq_ind, pol_ind]),
        np.imag(gains_init[:, freq_ind, pol_ind]),
    ),
    axis=0,
).flatten()

cost = calibration.cost_function_single_pol_wrapper(
    gains_init_flattened,
    Nants,
    Nbls,
    model_visibilities[:, :, freq_ind, pol_ind],
    data_visibilities[:, :, freq_ind, pol_ind],
    visibility_weights[:, :, freq_ind, pol_ind],
    gains_exp_mat_1,
    gains_exp_mat_2,
    0.01,
)
jac = calibration.jacobian_single_pol_wrapper(
    gains_init_flattened,
    Nants,
    Nbls,
    model_visibilities[:, :, freq_ind, pol_ind],
    data_visibilities[:, :, freq_ind, pol_ind],
    visibility_weights[:, :, freq_ind, pol_ind],
    gains_exp_mat_1,
    gains_exp_mat_2,
    0.01,
)
hess = calibration.hessian_single_pol_wrapper(
    gains_init_flattened,
    Nants,
    Nbls,
    model_visibilities[:, :, freq_ind, pol_ind],
    data_visibilities[:, :, freq_ind, pol_ind],
    visibility_weights[:, :, freq_ind, pol_ind],
    gains_exp_mat_1,
    gains_exp_mat_2,
    0.01,
)

In [ ]:
print(cost)

In [ ]:
print(jac)
print(hess)

In [ ]:
np.where(hess - np.conj(hess).T != 0)

In [ ]:
np.linalg.eigvals(np.conj(hess.T))

In [ ]:
hess - np.conj(hess).T

In [7]:
freq_ind = 0
pol_ind = 0
antenna_weights = np.sum(
    np.matmul(
        gains_exp_mat_1.T,
        visibility_weights[
            :,
            :,
            freq_ind,
            pol_ind,
        ].T,
    )
    + np.matmul(
        gains_exp_mat_2.T,
        visibility_weights[
            :,
            :,
            freq_ind,
            pol_ind,
        ].T,
    ),
    axis=1,
)

In [8]:
print(antenna_weights)

[  0.   0.   0.   0.   0.   0.   0.   0. 247.   0.   0.   0.   0.   0.
   0.   0.   0. 247.   0. 247. 247. 247.   0. 247.   0. 247.   0.   0.
 247.   0. 247.   0.   0. 247. 247.   0.   0.   0.   0.   0.   0.   0.
   0.   0. 247.   0. 247.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0. 247. 247. 247. 247. 247. 247. 247. 247. 247.
 247. 247. 247. 247. 247. 247. 247.   0. 247. 247. 247. 247. 247. 247.
 247. 247. 247. 247. 247. 247.   0. 247.   0. 247.   0.   0. 247. 247.
 247. 247. 247. 247. 247. 247. 247. 247. 247. 247. 247. 247. 247. 247.
 247. 247.   0. 247. 247. 247. 247. 247. 247. 247. 247.   0.   0.   0.
   0.   0. 247. 247. 247. 247. 247. 247. 247. 247. 247. 247. 247. 247.
 247. 247.   0. 247. 247. 247. 247. 247. 247. 247. 247.   0.   0. 247.
 247.   0.   0. 247.   0. 247. 247. 247. 247. 247. 247. 247. 247. 247.
 247. 247. 247. 247. 247. 247. 247. 247. 247. 247.   0. 247. 247. 247.
 247. 247. 247. 247. 247. 247.   0. 247.   0. 247. 247. 247. 247. 247.
 247. 